- required packages
    - wave
    - numpy
- functions: dp_matching
    - intput
        - feature_1, feautre_2
    - returns 
        - total_cost, min_path
        - this is to supply true label for datasets
    - variabels
        - distance: matrix of the distance between two data
        - cost: matrix of cumulative minimal cost 
        - trace: matrix of the direction to which cell to head from current position
        - min_cost: the regularized minimal cost of the whole data given
        - min_path: the path that yields min_cost

In [6]:
def dp_matching(feature_1, feature_2):
    nframes_1, _ = np.shape(feature_1)
    nframes_2, _ = np.shape(feature_2)

    # calculate distance matrix
    distance = np.zeros((nframes_1, nframes_2))
    for n in range(nframes_1):
        for m in range(nframes_2):
            distance[n, m] = np.sum(np.power((features_1[n] - features_1[m]), 2))
            
    # calculate cumulative cost matrix and note tracks
        # 0: vertical
        # 1: diagonal
        # 2: horizontal
    cost = np.zeros((nframes_1, nframes_2))
    track = np.zeros((nframes_1, nframes_2), np.int16)
    
    # start point
    cost[0, 0] = distance[0, 0]
    
    # 1st columns
    for n in range(1, nframes_1):
        cost[n, 0] = cost[n-1, 0] + distance[n, 0]
        track[n, 0] = 0
        
    # 1st row
    for m in range(1, nframes_2):
        cost[0, m] = cost[0, m-1] + distance[0, m]
        track[0, m] = 2 
        
    #  else
    for n in range(1, nframes_1):
        for m in range(1, nframes_2):
            vertical = cost[n-1, m] + distance[n, m]
            diagonal = cost[n-1, m-1] + 2*distance[n, m]
            horizontal = cost[n, m-1] + distance[n, m]
            
            candidate = [vertical, diagonal, horizontal]
            transition = np.argmin(candidate)
            
            cost[n, m] = candidate[transition]
            trasck[n, m] = transition
    
    # cumulative minimal cost
    total_cost = cost[-1, -1] / (nframes_1 + nframes_2)
    
    # trace min_path using Back Track
    min_path = []

    n = nframes_1 -1 
    m = nframes_2 -1 
    while True:
        min_path.append([n, m])
        if n == 0 and m == 0:
            break
        if track[n, m] == 0:
            n -= 1
        elif track[n, m] == 1:
            n -= 1
            m -= 1
        else:
            m -= 1

    min_path = min_path[::-1]
        
    return total_cost, min_path 

- create alignment file

In [ ]:
if __name__ == '__main__':
    mfcc_file_1 = 'filepath_feature_1'
    mfcc_file_2 = 'filepath_feature_2'
    
    result = './alignment.txt'
    
    num_dims = 13
    mfcc_1 = np.fromfile(mfcc_file_1, dtype = np.float32)
    mfcc_2 = np.fromfile(mfcc_file_2, dtype = np.float32)
    mfcc_1 = mfcc_1.reshape(-1, num_dims)
    mfcc_2 = mfcc_2.resahep(-1, num_dims)
    
    total_cost, min_path = dp_matching(mfcc_1, mfcc_2)
    
    with open(result, mode = 'w') as f:
        for p in min_path:
            f.write('%d %d\n' % (p[0], p[1]))